<a href="https://colab.research.google.com/github/abureyanahmed/GNN/blob/master/Steiner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gnn

In [2]:
!pip install tensorflow-gpu==1.15

     |████████████████████████████████| 411.5MB 37kB/s 
     |████████████████████████████████| 3.8MB 38.6MB/s 
     |████████████████████████████████| 512kB 49.4MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=7aa1077aaa3ca576dc5149474cf32cfec01ffb9c03658cbbd92c786b313ca310
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing instal

In [3]:
import tensorflow as tf
tf.version

<module 'tensorflow._api.v1.version' from '/usr/local/lib/python3.7/dist-packages/tensorflow_core/_api/v1/version/__init__.py'>

In [4]:
import numpy as np

In [5]:
def weight_variable(shape, nm):
    # function to initialize weights
    initial = tf.truncated_normal(shape, stddev=0.1)
    tf.summary.histogram(nm, initial, collections=['always'])
    return tf.Variable(initial, name=nm)



In [6]:
class Net:
    # class to define state and output network

    def __init__(self, input_dim, state_dim, output_dim):
        # initialize weight and parameter

        self.EPSILON = 0.00000001

        self.input_dim = input_dim
        self.state_dim = state_dim
        self.output_dim = output_dim
        self.state_input = self.input_dim - 1 + state_dim  # removing the id_ dimension

        #### TO BE SET ON A SPECIFIC PROBLEM
        self.state_l1 = 15
        self.state_l2 = self.state_dim

        self.output_l1 = 10
        self.output_l2 = self.output_dim

    def netSt(self, inp):
        with tf.variable_scope('State_net'):

            layer1 = tf.layers.dense(inp, self.state_l1, activation=tf.nn.tanh)
            layer2 = tf.layers.dense(layer1, self.state_l2, activation=tf.nn.tanh)

            return layer2

    def netOut(self, inp):

            layer1 = tf.layers.dense(inp, self.output_l1, activation=tf.nn.tanh)
            layer2 = tf.layers.dense(layer1, self.output_l2, activation=tf.nn.softmax)

            return layer2

    def Loss(self, output, target, output_weight=None):
        # method to define the loss function
        #lo = tf.losses.softmax_cross_entropy(target, output)
        output = tf.maximum(output, self.EPSILON, name="Avoiding_explosions")  # to avoid explosions
        xent = -tf.reduce_sum(target * tf.log(output), 1)
        lo = tf.reduce_mean(xent)
        return lo

    def Metric(self, target, output, output_weight=None):
        # method to define the evaluation metric
        correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(target, 1))
        metric = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        return metric



In [7]:
import scipy
import scipy.io as spio

In [8]:
sess = tf.Session()

In [9]:
import tensorflow as tf
import numpy as np
import  gnn.gnn_utils as gnn_utils
import gnn.GNN as GNN 


import networkx as nx
import scipy as sp

import matplotlib.pyplot as plt

##### GPU & stuff config
import os

os.environ['CUDA_VISIBLE_DEVICES'] = "0"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True


In [10]:
total_edges = [[0, 18, 0], [0, 37, 0], [0, 39, 0], [0, 41, 0], [0, 42, 0], [0, 11, 0], [0, 46, 0], [0, 15, 0], [1, 18, 0], [1, 20, 0], [1, 7, 0], [1, 40, 0], [1, 4, 0], [1, 45, 0], [2, 25, 0], [2, 6, 0], [2, 39, 0], [3, 21, 0], [3, 22, 0], [3, 47, 0], [3, 10, 0], [3, 46, 0], [3, 30, 0], [3, 15, 0], [4, 16, 0], [4, 17, 0], [4, 5, 0], [4, 23, 0], [4, 41, 0], [4, 30, 0], [4, 37, 0], [5, 34, 0], [5, 20, 0], [5, 37, 0], [5, 42, 0], [5, 43, 0], [6, 33, 0], [6, 20, 0], [6, 22, 0], [6, 40, 0], [6, 26, 0], [6, 42, 0], [7, 16, 0], [7, 35, 0], [7, 37, 0], [7, 40, 0], [7, 8, 0], [7, 25, 0], [7, 42, 0], [7, 28, 0], [7, 47, 0], [8, 16, 0], [8, 18, 0], [8, 20, 0], [8, 37, 0], [8, 39, 0], [9, 18, 0], [9, 39, 0], [9, 28, 0], [9, 29, 0], [9, 46, 0], [9, 47, 0], [10, 16, 0], [10, 28, 0], [10, 13, 0], [11, 26, 0], [11, 34, 0], [11, 38, 0], [11, 47, 0], [12, 34, 0], [12, 21, 0], [12, 38, 0], [12, 23, 0], [12, 27, 0], [12, 46, 0], [13, 32, 0], [13, 40, 0], [13, 37, 0], [13, 19, 0], [13, 21, 0], [13, 24, 0], [13, 26, 0], [13, 15, 0], [14, 24, 0], [14, 35, 0], [14, 19, 0], [14, 47, 0], [15, 36, 0], [15, 38, 0], [15, 39, 0], [15, 26, 0], [15, 28, 0], [16, 32, 0], [16, 17, 0], [16, 19, 0], [16, 38, 0], [16, 41, 0], [16, 39, 0], [17, 49, 0], [17, 38, 0], [17, 24, 0], [17, 26, 0], [18, 35, 0], [18, 36, 0], [18, 41, 0], [18, 26, 0], [18, 29, 0], [18, 46, 0], [18, 31, 0], [19, 30, 0], [19, 22, 0], [19, 46, 0], [19, 28, 0], [20, 22, 0], [20, 39, 0], [20, 30, 0], [21, 32, 0], [21, 34, 0], [21, 27, 0], [21, 28, 0], [21, 29, 0], [22, 32, 0], [22, 39, 0], [22, 24, 0], [22, 27, 0], [23, 32, 0], [23, 30, 0], [23, 31, 0], [24, 38, 0], [24, 40, 0], [24, 42, 0], [24, 46, 0], [24, 44, 0], [24, 47, 0], [25, 44, 0], [26, 27, 0], [26, 33, 0], [27, 36, 0], [27, 38, 0], [27, 47, 0], [28, 38, 0], [28, 39, 0], [29, 37, 0], [29, 39, 0], [29, 31, 0], [30, 45, 0], [32, 47, 0], [33, 48, 0], [33, 36, 0], [33, 38, 0], [33, 43, 0], [34, 48, 0], [34, 47, 0], [35, 41, 0], [35, 49, 0], [36, 47, 0], [38, 44, 0], [38, 39, 0], [39, 44, 0], [39, 49, 0], [39, 47, 0], [41, 49, 0], [41, 42, 0], [42, 45, 0], [43, 49, 0], [45, 48, 0], [45, 49, 0], [45, 47, 0], [46, 47, 0], [48, 49, 0], [50, 90, 1], [50, 51, 1], [50, 84, 1], [50, 83, 1], [50, 57, 1], [51, 84, 1], [51, 53, 1], [51, 72, 1], [51, 94, 1], [51, 62, 1], [51, 79, 1], [51, 95, 1], [52, 78, 1], [52, 79, 1], [52, 73, 1], [53, 85, 1], [53, 87, 1], [53, 88, 1], [53, 57, 1], [53, 61, 1], [53, 94, 1], [53, 65, 1], [53, 70, 1], [53, 71, 1], [53, 74, 1], [53, 77, 1], [53, 79, 1], [54, 76, 1], [54, 78, 1], [54, 96, 1], [54, 88, 1], [55, 98, 1], [55, 69, 1], [55, 71, 1], [55, 90, 1], [55, 92, 1], [55, 78, 1], [55, 80, 1], [56, 87, 1], [56, 72, 1], [56, 89, 1], [56, 58, 1], [56, 59, 1], [56, 76, 1], [57, 66, 1], [57, 84, 1], [57, 70, 1], [57, 67, 1], [57, 78, 1], [57, 89, 1], [57, 62, 1], [57, 96, 1], [57, 86, 1], [58, 82, 1], [58, 83, 1], [58, 84, 1], [58, 89, 1], [58, 90, 1], [58, 60, 1], [58, 62, 1], [58, 92, 1], [58, 70, 1], [58, 79, 1], [58, 80, 1], [59, 74, 1], [59, 81, 1], [60, 67, 1], [60, 70, 1], [60, 90, 1], [60, 83, 1], [60, 78, 1], [60, 63, 1], [60, 96, 1], [61, 98, 1], [61, 69, 1], [61, 80, 1], [61, 92, 1], [61, 96, 1], [62, 85, 1], [62, 75, 1], [62, 97, 1], [62, 78, 1], [62, 81, 1], [63, 75, 1], [63, 94, 1], [63, 73, 1], [64, 66, 1], [64, 74, 1], [64, 77, 1], [64, 94, 1], [65, 66, 1], [65, 68, 1], [65, 69, 1], [65, 91, 1], [65, 76, 1], [65, 79, 1], [65, 80, 1], [65, 97, 1], [66, 80, 1], [66, 96, 1], [66, 78, 1], [66, 79, 1], [67, 83, 1], [67, 70, 1], [67, 89, 1], [67, 72, 1], [67, 73, 1], [67, 75, 1], [67, 80, 1], [68, 69, 1], [68, 70, 1], [68, 90, 1], [68, 92, 1], [68, 94, 1], [69, 90, 1], [69, 71, 1], [69, 92, 1], [69, 95, 1], [69, 96, 1], [70, 75, 1], [71, 83, 1], [71, 87, 1], [72, 86, 1], [72, 87, 1], [72, 99, 1], [73, 88, 1], [73, 89, 1], [73, 91, 1], [73, 92, 1], [73, 77, 1], [73, 81, 1], [74, 83, 1], [74, 86, 1], [74, 87, 1], [74, 93, 1], [74, 75, 1], [74, 89, 1], [74, 95, 1], [75, 78, 1], [76, 85, 1], [76, 87, 1], [77, 99, 1], [77, 96, 1], [77, 88, 1], [77, 89, 1], [77, 95, 1], [78, 88, 1], [78, 93, 1], [78, 92, 1], [79, 98, 1], [79, 88, 1], [79, 90, 1], [79, 91, 1], [80, 93, 1], [80, 96, 1], [81, 93, 1], [81, 85, 1], [81, 87, 1], [81, 89, 1], [84, 87, 1], [84, 92, 1], [84, 96, 1], [85, 89, 1], [85, 97, 1], [86, 90, 1], [86, 94, 1], [87, 92, 1], [88, 96, 1], [89, 95, 1], [90, 92, 1], [90, 99, 1], [91, 97, 1], [91, 94, 1], [100, 101, 2], [100, 138, 2], [100, 139, 2], [100, 143, 2], [100, 145, 2], [100, 146, 2], [101, 120, 2], [101, 146, 2], [101, 139, 2], [101, 140, 2], [101, 136, 2], [101, 110, 2], [101, 112, 2], [101, 114, 2], [101, 147, 2], [102, 125, 2], [102, 147, 2], [102, 113, 2], [102, 133, 2], [103, 132, 2], [103, 138, 2], [103, 106, 2], [103, 123, 2], [103, 108, 2], [103, 141, 2], [103, 144, 2], [103, 114, 2], [103, 131, 2], [104, 148, 2], [104, 133, 2], [104, 105, 2], [104, 123, 2], [104, 146, 2], [104, 109, 2], [104, 110, 2], [104, 111, 2], [104, 128, 2], [104, 113, 2], [104, 114, 2], [105, 126, 2], [105, 143, 2], [106, 137, 2], [106, 140, 2], [106, 109, 2], [106, 113, 2], [106, 121, 2], [107, 125, 2], [107, 118, 2], [107, 141, 2], [107, 133, 2], [108, 132, 2], [108, 149, 2], [108, 140, 2], [108, 141, 2], [108, 113, 2], [108, 117, 2], [108, 109, 2], [108, 123, 2], [108, 124, 2], [108, 127, 2], [108, 131, 2], [109, 133, 2], [109, 143, 2], [109, 137, 2], [109, 141, 2], [109, 120, 2], [109, 111, 2], [109, 113, 2], [110, 133, 2], [111, 116, 2], [111, 117, 2], [111, 149, 2], [111, 121, 2], [111, 125, 2], [111, 141, 2], [111, 128, 2], [111, 114, 2], [111, 136, 2], [112, 148, 2], [112, 149, 2], [112, 123, 2], [112, 141, 2], [112, 131, 2], [113, 117, 2], [113, 138, 2], [113, 139, 2], [113, 127, 2], [113, 146, 2], [114, 134, 2], [114, 139, 2], [114, 140, 2], [115, 119, 2], [115, 135, 2], [115, 124, 2], [115, 143, 2], [115, 146, 2], [115, 147, 2], [116, 132, 2], [116, 133, 2], [116, 138, 2], [116, 124, 2], [116, 127, 2], [116, 128, 2], [116, 129, 2], [116, 131, 2], [118, 119, 2], [118, 128, 2], [120, 135, 2], [121, 132, 2], [121, 131, 2], [122, 124, 2], [122, 128, 2], [123, 143, 2], [123, 136, 2], [123, 142, 2], [123, 127, 2], [123, 145, 2], [123, 131, 2], [124, 139, 2], [124, 144, 2], [125, 132, 2], [125, 141, 2], [125, 142, 2], [125, 134, 2], [126, 131, 2], [127, 143, 2], [127, 131, 2], [128, 148, 2], [128, 135, 2], [128, 149, 2], [128, 142, 2], [128, 130, 2], [129, 143, 2], [129, 135, 2], [129, 139, 2], [130, 132, 2], [130, 134, 2], [130, 140, 2], [130, 141, 2], [130, 143, 2], [131, 143, 2], [131, 141, 2], [131, 142, 2], [132, 148, 2], [132, 143, 2], [133, 144, 2], [134, 138, 2], [135, 148, 2], [135, 139, 2], [136, 137, 2], [136, 139, 2], [136, 142, 2], [137, 145, 2], [139, 148, 2], [139, 141, 2], [141, 142, 2], [141, 144, 2], [142, 149, 2], [142, 143, 2], [142, 145, 2], [142, 147, 2], [143, 149, 2], [144, 149, 2], [145, 148, 2], [148, 149, 2], [150, 199, 3], [150, 154, 3], [150, 187, 3], [150, 156, 3], [151, 198, 3], [151, 153, 3], [151, 172, 3], [151, 173, 3], [151, 160, 3], [151, 164, 3], [151, 181, 3], [152, 183, 3], [152, 153, 3], [152, 178, 3], [152, 159, 3], [152, 177, 3], [152, 162, 3], [152, 164, 3], [153, 154, 3], [153, 183, 3], [153, 190, 3], [153, 193, 3], [153, 162, 3], [153, 164, 3], [153, 198, 3], [153, 199, 3], [153, 174, 3], [153, 189, 3], [153, 180, 3], [154, 167, 3], [154, 156, 3], [154, 198, 3], [154, 197, 3], [154, 169, 3], [154, 196, 3], [154, 165, 3], [155, 171, 3], [155, 188, 3], [155, 174, 3], [155, 176, 3], [155, 162, 3], [155, 192, 3], [155, 187, 3], [156, 166, 3], [156, 168, 3], [156, 169, 3], [156, 184, 3], [156, 179, 3], [156, 165, 3], [157, 198, 3], [157, 171, 3], [157, 172, 3], [157, 178, 3], [157, 176, 3], [157, 194, 3], [157, 195, 3], [158, 190, 3], [158, 167, 3], [158, 162, 3], [158, 164, 3], [158, 173, 3], [159, 166, 3], [159, 170, 3], [159, 171, 3], [159, 172, 3], [159, 191, 3], [159, 192, 3], [159, 163, 3], [160, 182, 3], [160, 184, 3], [160, 186, 3], [160, 187, 3], [160, 172, 3], [160, 175, 3], [160, 194, 3], [160, 163, 3], [161, 185, 3], [161, 169, 3], [161, 172, 3], [161, 174, 3], [161, 192, 3], [161, 177, 3], [162, 188, 3], [162, 189, 3], [162, 178, 3], [162, 187, 3], [163, 185, 3], [163, 169, 3], [163, 189, 3], [163, 190, 3], [163, 192, 3], [163, 181, 3], [164, 197, 3], [165, 198, 3], [165, 171, 3], [165, 172, 3], [165, 191, 3], [165, 190, 3], [165, 175, 3], [165, 194, 3], [165, 179, 3], [165, 187, 3], [166, 199, 3], [166, 170, 3], [166, 171, 3], [166, 190, 3], [166, 193, 3], [166, 196, 3], [166, 186, 3], [167, 191, 3], [167, 193, 3], [167, 178, 3], [167, 179, 3], [168, 198, 3], [168, 190, 3], [168, 169, 3], [168, 174, 3], [168, 191, 3], [168, 176, 3], [168, 178, 3], [168, 194, 3], [169, 198, 3], [169, 184, 3], [169, 187, 3], [169, 176, 3], [170, 199, 3], [170, 187, 3], [170, 173, 3], [170, 193, 3], [170, 197, 3], [171, 182, 3], [171, 195, 3], [171, 185, 3], [171, 183, 3], [171, 193, 3], [171, 192, 3], [172, 187, 3], [172, 195, 3], [172, 196, 3], [172, 173, 3], [172, 197, 3], [172, 180, 3], [172, 181, 3], [173, 197, 3], [173, 181, 3], [174, 185, 3], [174, 188, 3], [174, 194, 3], [175, 183, 3], [175, 189, 3], [176, 182, 3], [176, 183, 3], [176, 186, 3], [176, 192, 3], [176, 184, 3], [176, 180, 3], [177, 193, 3], [177, 184, 3], [177, 181, 3], [178, 186, 3], [178, 194, 3], [178, 189, 3], [178, 197, 3], [179, 185, 3], [179, 190, 3], [179, 195, 3], [179, 196, 3], [180, 186, 3], [180, 194, 3], [180, 196, 3], [180, 181, 3], [181, 190, 3], [181, 194, 3], [181, 197, 3], [183, 189, 3], [184, 185, 3], [184, 196, 3], [184, 197, 3], [185, 198, 3], [185, 187, 3], [186, 190, 3], [188, 189, 3], [190, 198, 3], [190, 191, 3], [193, 198, 3], [193, 195, 3], [194, 199, 3], [194, 197, 3], [196, 198, 3], [196, 199, 3], [200, 246, 4], [200, 229, 4], [200, 222, 4], [200, 239, 4], [201, 232, 4], [201, 211, 4], [201, 238, 4], [201, 222, 4], [201, 223, 4], [201, 244, 4], [201, 227, 4], [201, 212, 4], [202, 217, 4], [202, 219, 4], [202, 221, 4], [202, 243, 4], [202, 229, 4], [202, 214, 4], [203, 240, 4], [203, 233, 4], [203, 237, 4], [203, 222, 4], [203, 223, 4], [204, 216, 4], [204, 209, 4], [204, 210, 4], [204, 235, 4], [204, 234, 4], [205, 217, 4], [205, 219, 4], [205, 223, 4], [205, 226, 4], [205, 207, 4], [205, 229, 4], [205, 214, 4], [206, 248, 4], [206, 207, 4], [206, 208, 4], [206, 226, 4], [206, 213, 4], [206, 214, 4], [206, 229, 4], [207, 218, 4], [207, 236, 4], [207, 224, 4], [207, 247, 4], [207, 234, 4], [207, 230, 4], [207, 245, 4], [208, 232, 4], [208, 248, 4], [208, 218, 4], [208, 221, 4], [208, 225, 4], [209, 242, 4], [209, 222, 4], [209, 241, 4], [209, 226, 4], [209, 229, 4], [210, 248, 4], [210, 235, 4], [210, 240, 4], [210, 241, 4], [210, 242, 4], [210, 243, 4], [210, 213, 4], [211, 240, 4], [211, 237, 4], [211, 225, 4], [212, 248, 4], [212, 219, 4], [212, 238, 4], [212, 223, 4], [212, 224, 4], [212, 226, 4], [212, 244, 4], [212, 213, 4], [212, 247, 4], [213, 233, 4], [213, 240, 4], [213, 244, 4], [213, 247, 4], [213, 220, 4], [213, 223, 4], [213, 226, 4], [213, 228, 4], [213, 231, 4], [214, 216, 4], [214, 239, 4], [214, 241, 4], [214, 227, 4], [214, 244, 4], [214, 245, 4], [214, 247, 4], [215, 217, 4], [215, 226, 4], [215, 238, 4], [216, 249, 4], [216, 239, 4], [216, 227, 4], [216, 230, 4], [216, 237, 4], [217, 235, 4], [217, 241, 4], [217, 242, 4], [217, 243, 4], [217, 244, 4], [217, 245, 4], [217, 220, 4], [217, 224, 4], [217, 247, 4], [217, 229, 4], [217, 231, 4], [218, 240, 4], [218, 242, 4], [218, 228, 4], [219, 233, 4], [219, 240, 4], [219, 241, 4], [220, 242, 4], [220, 237, 4], [220, 238, 4], [220, 240, 4], [220, 226, 4], [220, 244, 4], [220, 230, 4], [221, 224, 4], [221, 236, 4], [221, 233, 4], [221, 222, 4], [221, 223, 4], [221, 242, 4], [221, 230, 4], [222, 248, 4], [222, 239, 4], [222, 226, 4], [222, 229, 4], [222, 245, 4], [223, 237, 4], [223, 238, 4], [223, 249, 4], [223, 228, 4], [223, 246, 4], [224, 236, 4], [224, 240, 4], [224, 245, 4], [224, 237, 4], [225, 242, 4], [225, 245, 4], [225, 230, 4], [226, 232, 4], [226, 242, 4], [226, 247, 4], [226, 228, 4], [226, 244, 4], [226, 230, 4], [226, 237, 4], [227, 236, 4], [227, 240, 4], [227, 241, 4], [227, 245, 4], [227, 231, 4], [228, 248, 4], [228, 233, 4], [228, 249, 4], [228, 246, 4], [228, 243, 4], [228, 230, 4], [229, 242, 4], [230, 234, 4], [231, 242, 4], [231, 244, 4], [232, 236, 4], [232, 249, 4], [232, 233, 4], [232, 243, 4], [232, 247, 4], [233, 244, 4], [233, 247, 4], [234, 248, 4], [234, 249, 4], [234, 246, 4], [235, 239, 4], [236, 248, 4], [236, 238, 4], [236, 241, 4], [237, 240, 4], [237, 248, 4], [237, 249, 4], [239, 248, 4], [239, 243, 4], [240, 243, 4], [240, 246, 4], [241, 246, 4], [242, 244, 4], [242, 246, 4], [245, 248, 4]]
total_nodes = [[0, 1], [1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [6, 1], [7, 1], [8, 1], [9, 1], [10, 1], [11, 1], [12, 1], [13, 1], [14, 1], [15, 1], [16, 1], [17, 1], [18, 1], [19, 1], [20, 1], [21, 1], [22, 1], [23, 1], [24, 1], [25, 0], [26, 0], [27, 0], [28, 0], [29, 0], [30, 0], [31, 0], [32, 0], [33, 0], [34, 0], [35, 0], [36, 0], [37, 0], [38, 0], [39, 0], [40, 0], [41, 0], [42, 0], [43, 0], [44, 0], [45, 0], [46, 0], [47, 0], [48, 0], [49, 0], [50, 1], [51, 1], [52, 1], [53, 1], [54, 1], [55, 1], [56, 1], [57, 1], [58, 1], [59, 1], [60, 1], [61, 1], [62, 1], [63, 1], [64, 1], [65, 1], [66, 1], [67, 1], [68, 1], [69, 1], [70, 1], [71, 1], [72, 1], [73, 1], [74, 1], [75, 0], [76, 0], [77, 0], [78, 0], [79, 0], [80, 0], [81, 0], [82, 0], [83, 0], [84, 0], [85, 0], [86, 0], [87, 0], [88, 0], [89, 0], [90, 0], [91, 0], [92, 0], [93, 0], [94, 0], [95, 0], [96, 0], [97, 0], [98, 0], [99, 0], [100, 1], [101, 1], [102, 1], [103, 1], [104, 1], [105, 1], [106, 1], [107, 1], [108, 1], [109, 1], [110, 1], [111, 1], [112, 1], [113, 1], [114, 1], [115, 1], [116, 1], [117, 1], [118, 1], [119, 1], [120, 1], [121, 1], [122, 1], [123, 1], [124, 1], [125, 0], [126, 0], [127, 0], [128, 0], [129, 0], [130, 0], [131, 0], [132, 0], [133, 0], [134, 0], [135, 0], [136, 0], [137, 0], [138, 0], [139, 0], [140, 0], [141, 0], [142, 0], [143, 0], [144, 0], [145, 0], [146, 0], [147, 0], [148, 0], [149, 0], [150, 1], [151, 1], [152, 1], [153, 1], [154, 1], [155, 1], [156, 1], [157, 1], [158, 1], [159, 1], [160, 1], [161, 1], [162, 1], [163, 1], [164, 1], [165, 1], [166, 1], [167, 1], [168, 1], [169, 1], [170, 1], [171, 1], [172, 1], [173, 1], [174, 1], [175, 0], [176, 0], [177, 0], [178, 0], [179, 0], [180, 0], [181, 0], [182, 0], [183, 0], [184, 0], [185, 0], [186, 0], [187, 0], [188, 0], [189, 0], [190, 0], [191, 0], [192, 0], [193, 0], [194, 0], [195, 0], [196, 0], [197, 0], [198, 0], [199, 0], [200, 1], [201, 1], [202, 1], [203, 1], [204, 1], [205, 1], [206, 1], [207, 1], [208, 1], [209, 1], [210, 1], [211, 1], [212, 1], [213, 1], [214, 1], [215, 1], [216, 1], [217, 1], [218, 1], [219, 1], [220, 1], [221, 1], [222, 1], [223, 1], [224, 1], [225, 0], [226, 0], [227, 0], [228, 0], [229, 0], [230, 0], [231, 0], [232, 0], [233, 0], [234, 0], [235, 0], [236, 0], [237, 0], [238, 0], [239, 0], [240, 0], [241, 0], [242, 0], [243, 0], [244, 0], [245, 0], [246, 0], [247, 0], [248, 0], [249, 0]]


In [11]:
E_tot = np.asarray(total_edges)
N_tot = np.asarray(total_nodes)

In [20]:
total_labels = [[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]

In [21]:
labels = np.asarray(total_labels)

In [22]:
inp, arcnode, graphnode = gnn_utils.from_EN_to_GNN(E_tot, N_tot)

In [23]:
threshold = 0.01
learning_rate = 0.01
state_dim = 5
tf.reset_default_graph()
input_dim = inp.shape[1]
output_dim = 1
max_it = 50
num_epoch = 10000
optimizer = tf.train.AdamOptimizer
# initialize state and output network
net = Net(input_dim, state_dim, output_dim)

In [24]:
param = "st_d" + str(state_dim) + "_th" + str(threshold) + "_lr" + str(learning_rate)
print(param)

st_d5_th0.01_lr0.01


In [25]:
tensorboard = False

g = GNN.GNN(net, input_dim, output_dim, state_dim,  max_it, optimizer, learning_rate, threshold, graph_based=False, param=param, config=config,
            tensorboard=tensorboard)

/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [27]:
# train the model
count = 0

######
print(inp.shape)
print(labels.shape)
for j in range(0, num_epoch):
    _, it = g.Train(inputs=inp, ArcNode=arcnode, target=labels, step=count)
    if count % 30 == 0:
        print("Epoch ", count)
        print("Training: ", g.Validate(inp, arcnode, labels, count))

        # end = time.time()
        # print("Epoch {} at time {}".format(j, end-start))
        # start = time.time()

    count = count + 1


(895, 4)
(250, 1)
Epoch  0
Training:  (0.0, 1.0, 5)
Epoch  30
Training:  (0.0, 1.0, 5)
Epoch  60
Training:  (0.0, 1.0, 5)
Epoch  90
Training:  (0.0, 1.0, 5)
Epoch  120
Training:  (0.0, 1.0, 5)
Epoch  150
Training:  (0.0, 1.0, 5)
Epoch  180
Training:  (0.0, 1.0, 5)
Epoch  210
Training:  (0.0, 1.0, 5)
Epoch  240
Training:  (0.0, 1.0, 5)
Epoch  270
Training:  (0.0, 1.0, 5)
Epoch  300
Training:  (0.0, 1.0, 5)
Epoch  330
Training:  (0.0, 1.0, 5)
Epoch  360
Training:  (0.0, 1.0, 5)
Epoch  390
Training:  (0.0, 1.0, 5)
Epoch  420
Training:  (0.0, 1.0, 5)
Epoch  450
Training:  (0.0, 1.0, 5)
Epoch  480
Training:  (0.0, 1.0, 5)
Epoch  510
Training:  (0.0, 1.0, 5)
Epoch  540
Training:  (0.0, 1.0, 5)
Epoch  570
Training:  (0.0, 1.0, 5)
Epoch  600
Training:  (0.0, 1.0, 5)
Epoch  630
Training:  (0.0, 1.0, 5)
Epoch  660
Training:  (0.0, 1.0, 5)
Epoch  690
Training:  (0.0, 1.0, 5)
Epoch  720
Training:  (0.0, 1.0, 5)
Epoch  750
Training:  (0.0, 1.0, 5)
Epoch  780
Training:  (0.0, 1.0, 5)
Epoch  810
Trai

In [28]:
met = g.Evaluate(inp, arcnode, labels)
met

[1.0]

In [29]:
test_total_edges = [[0, 48, 0], [0, 39, 0], [0, 41, 0], [0, 10, 0], [0, 12, 0], [0, 46, 0], [0, 15, 0], [0, 16, 0], [0, 14, 0], [0, 22, 0], [0, 25, 0], [0, 27, 0], [1, 32, 0], [1, 7, 0], [1, 25, 0], [1, 10, 0], [1, 42, 0], [1, 30, 0], [1, 15, 0], [2, 40, 0], [2, 33, 0], [2, 10, 0], [2, 45, 0], [2, 22, 0], [3, 16, 0], [3, 20, 0], [3, 12, 0], [3, 48, 0], [3, 44, 0], [4, 32, 0], [4, 38, 0], [4, 23, 0], [4, 24, 0], [4, 41, 0], [4, 27, 0], [4, 28, 0], [4, 45, 0], [5, 40, 0], [5, 23, 0], [5, 12, 0], [5, 45, 0], [5, 15, 0], [6, 48, 0], [6, 17, 0], [6, 20, 0], [6, 38, 0], [6, 22, 0], [6, 7, 0], [6, 9, 0], [6, 12, 0], [6, 29, 0], [6, 14, 0], [7, 27, 0], [7, 36, 0], [7, 37, 0], [8, 32, 0], [8, 18, 0], [8, 21, 0], [8, 38, 0], [8, 42, 0], [8, 28, 0], [8, 46, 0], [8, 37, 0], [9, 16, 0], [9, 33, 0], [9, 49, 0], [9, 24, 0], [9, 25, 0], [9, 27, 0], [9, 29, 0], [9, 30, 0], [10, 44, 0], [11, 32, 0], [11, 48, 0], [11, 35, 0], [11, 15, 0], [11, 12, 0], [11, 47, 0], [12, 43, 0], [12, 34, 0], [12, 36, 0], [12, 39, 0], [12, 24, 0], [12, 42, 0], [13, 49, 0], [13, 26, 0], [13, 42, 0], [13, 22, 0], [13, 14, 0], [14, 32, 0], [14, 34, 0], [14, 25, 0], [14, 43, 0], [14, 28, 0], [15, 43, 0], [15, 36, 0], [15, 38, 0], [15, 32, 0], [15, 25, 0], [15, 47, 0], [16, 17, 0], [16, 20, 0], [16, 43, 0], [16, 27, 0], [17, 48, 0], [17, 41, 0], [17, 46, 0], [17, 49, 0], [17, 20, 0], [17, 24, 0], [17, 25, 0], [17, 26, 0], [17, 29, 0], [17, 30, 0], [18, 42, 0], [18, 30, 0], [18, 47, 0], [19, 49, 0], [19, 20, 0], [19, 23, 0], [19, 25, 0], [19, 43, 0], [19, 31, 0], [20, 32, 0], [20, 33, 0], [20, 38, 0], [20, 46, 0], [20, 24, 0], [20, 37, 0], [20, 31, 0], [21, 35, 0], [21, 37, 0], [21, 45, 0], [22, 48, 0], [22, 35, 0], [22, 37, 0], [22, 49, 0], [22, 28, 0], [23, 48, 0], [23, 33, 0], [23, 49, 0], [23, 39, 0], [23, 24, 0], [23, 40, 0], [23, 45, 0], [24, 48, 0], [24, 30, 0], [24, 27, 0], [24, 44, 0], [25, 32, 0], [25, 35, 0], [26, 48, 0], [26, 43, 0], [26, 33, 0], [26, 39, 0], [26, 31, 0], [27, 32, 0], [27, 37, 0], [27, 42, 0], [27, 43, 0], [27, 41, 0], [27, 28, 0], [27, 29, 0], [28, 48, 0], [28, 33, 0], [28, 34, 0], [28, 35, 0], [28, 47, 0], [29, 41, 0], [29, 40, 0], [29, 46, 0], [30, 35, 0], [30, 41, 0], [30, 44, 0], [31, 40, 0], [31, 35, 0], [32, 38, 0], [32, 39, 0], [32, 49, 0], [33, 35, 0], [34, 48, 0], [34, 35, 0], [34, 36, 0], [34, 39, 0], [34, 40, 0], [35, 47, 0], [35, 48, 0], [35, 49, 0], [36, 46, 0], [37, 43, 0], [37, 41, 0], [38, 48, 0], [38, 49, 0], [39, 48, 0], [39, 47, 0], [40, 48, 0], [40, 49, 0], [41, 45, 0], [43, 44, 0], [50, 66, 1], [50, 72, 1], [50, 57, 1], [50, 59, 1], [50, 60, 1], [50, 93, 1], [50, 78, 1], [50, 95, 1], [50, 94, 1], [51, 66, 1], [51, 52, 1], [51, 53, 1], [51, 71, 1], [51, 59, 1], [51, 94, 1], [51, 81, 1], [52, 66, 1], [52, 68, 1], [52, 69, 1], [52, 86, 1], [52, 87, 1], [52, 61, 1], [52, 84, 1], [53, 66, 1], [53, 69, 1], [53, 99, 1], [53, 72, 1], [53, 98, 1], [53, 90, 1], [53, 76, 1], [53, 62, 1], [54, 99, 1], [54, 85, 1], [54, 56, 1], [54, 60, 1], [54, 76, 1], [54, 63, 1], [55, 99, 1], [55, 74, 1], [55, 81, 1], [55, 60, 1], [55, 77, 1], [55, 78, 1], [55, 63, 1], [55, 65, 1], [56, 68, 1], [56, 87, 1], [56, 95, 1], [57, 76, 1], [57, 64, 1], [57, 97, 1], [58, 67, 1], [58, 76, 1], [59, 98, 1], [59, 87, 1], [59, 83, 1], [59, 60, 1], [59, 96, 1], [59, 94, 1], [59, 80, 1], [60, 85, 1], [60, 69, 1], [60, 61, 1], [60, 94, 1], [60, 96, 1], [60, 65, 1], [61, 99, 1], [61, 84, 1], [61, 71, 1], [61, 89, 1], [61, 76, 1], [61, 78, 1], [61, 63, 1], [61, 65, 1], [62, 98, 1], [62, 83, 1], [62, 93, 1], [62, 94, 1], [62, 79, 1], [62, 80, 1], [63, 93, 1], [63, 94, 1], [63, 79, 1], [63, 64, 1], [64, 82, 1], [64, 83, 1], [64, 68, 1], [64, 71, 1], [64, 94, 1], [64, 77, 1], [64, 78, 1], [64, 96, 1], [64, 97, 1], [65, 99, 1], [65, 72, 1], [65, 91, 1], [65, 76, 1], [65, 94, 1], [66, 87, 1], [66, 83, 1], [66, 90, 1], [66, 78, 1], [66, 84, 1], [66, 81, 1], [67, 82, 1], [67, 83, 1], [67, 70, 1], [67, 88, 1], [67, 72, 1], [68, 98, 1], [68, 85, 1], [68, 69, 1], [68, 87, 1], [68, 92, 1], [68, 96, 1], [68, 81, 1], [69, 83, 1], [69, 99, 1], [69, 90, 1], [69, 94, 1], [69, 78, 1], [70, 99, 1], [70, 72, 1], [70, 73, 1], [70, 94, 1], [70, 77, 1], [70, 78, 1], [71, 83, 1], [71, 73, 1], [71, 90, 1], [71, 75, 1], [71, 78, 1], [71, 94, 1], [72, 91, 1], [72, 98, 1], [72, 85, 1], [72, 75, 1], [72, 76, 1], [72, 80, 1], [72, 81, 1], [73, 83, 1], [73, 89, 1], [73, 90, 1], [73, 92, 1], [73, 78, 1], [73, 81, 1], [74, 92, 1], [74, 99, 1], [74, 85, 1], [74, 86, 1], [74, 76, 1], [74, 77, 1], [74, 78, 1], [74, 79, 1], [74, 97, 1], [75, 91, 1], [75, 88, 1], [75, 78, 1], [76, 83, 1], [76, 89, 1], [76, 95, 1], [76, 85, 1], [76, 97, 1], [77, 98, 1], [77, 83, 1], [77, 89, 1], [78, 82, 1], [78, 92, 1], [79, 82, 1], [79, 83, 1], [79, 96, 1], [79, 80, 1], [80, 83, 1], [80, 88, 1], [80, 99, 1], [80, 96, 1], [80, 81, 1], [81, 88, 1], [81, 94, 1], [82, 98, 1], [82, 99, 1], [82, 89, 1], [82, 92, 1], [83, 86, 1], [83, 88, 1], [83, 94, 1], [84, 93, 1], [84, 87, 1], [84, 92, 1], [84, 95, 1], [84, 96, 1], [85, 96, 1], [86, 96, 1], [87, 93, 1], [87, 96, 1], [89, 96, 1], [92, 97, 1], [93, 94, 1], [93, 97, 1], [94, 95, 1], [94, 98, 1], [97, 99, 1], [100, 127, 2], [100, 109, 2], [100, 102, 2], [100, 111, 2], [100, 123, 2], [101, 102, 2], [101, 127, 2], [101, 136, 2], [101, 121, 2], [101, 106, 2], [101, 107, 2], [101, 146, 2], [101, 109, 2], [101, 111, 2], [101, 130, 2], [101, 137, 2], [102, 120, 2], [102, 133, 2], [102, 122, 2], [102, 117, 2], [102, 125, 2], [102, 148, 2], [102, 111, 2], [102, 129, 2], [102, 136, 2], [103, 119, 2], [103, 120, 2], [103, 121, 2], [103, 123, 2], [103, 140, 2], [103, 136, 2], [103, 112, 2], [103, 147, 2], [104, 140, 2], [104, 138, 2], [104, 107, 2], [104, 124, 2], [104, 109, 2], [104, 142, 2], [104, 111, 2], [104, 144, 2], [104, 114, 2], [105, 108, 2], [105, 116, 2], [105, 143, 2], [105, 124, 2], [106, 118, 2], [106, 121, 2], [106, 133, 2], [106, 141, 2], [106, 128, 2], [106, 147, 2], [107, 118, 2], [107, 141, 2], [107, 117, 2], [107, 120, 2], [107, 130, 2], [107, 147, 2], [108, 116, 2], [108, 117, 2], [108, 122, 2], [108, 125, 2], [108, 109, 2], [108, 130, 2], [108, 147, 2], [108, 114, 2], [108, 137, 2], [109, 143, 2], [109, 120, 2], [109, 126, 2], [109, 111, 2], [109, 129, 2], [110, 148, 2], [110, 117, 2], [110, 145, 2], [110, 140, 2], [110, 147, 2], [110, 112, 2], [110, 129, 2], [110, 131, 2], [111, 133, 2], [111, 145, 2], [111, 147, 2], [111, 149, 2], [111, 120, 2], [111, 121, 2], [111, 127, 2], [111, 130, 2], [112, 142, 2], [112, 148, 2], [112, 125, 2], [112, 126, 2], [112, 131, 2], [113, 116, 2], [113, 149, 2], [113, 134, 2], [113, 135, 2], [113, 137, 2], [113, 122, 2], [113, 140, 2], [113, 138, 2], [113, 115, 2], [114, 148, 2], [114, 140, 2], [114, 119, 2], [114, 149, 2], [114, 138, 2], [114, 123, 2], [114, 126, 2], [114, 128, 2], [114, 146, 2], [115, 132, 2], [115, 143, 2], [115, 141, 2], [115, 126, 2], [115, 127, 2], [116, 117, 2], [116, 118, 2], [116, 122, 2], [116, 149, 2], [116, 147, 2], [116, 126, 2], [116, 137, 2], [116, 131, 2], [117, 129, 2], [117, 130, 2], [118, 132, 2], [118, 119, 2], [118, 123, 2], [118, 125, 2], [118, 128, 2], [119, 121, 2], [119, 147, 2], [120, 138, 2], [120, 144, 2], [120, 134, 2], [120, 145, 2], [120, 148, 2], [120, 125, 2], [120, 126, 2], [120, 128, 2], [120, 130, 2], [120, 131, 2], [121, 132, 2], [121, 123, 2], [121, 139, 2], [121, 143, 2], [121, 129, 2], [121, 135, 2], [122, 132, 2], [122, 149, 2], [122, 123, 2], [123, 149, 2], [123, 124, 2], [123, 125, 2], [123, 138, 2], [124, 132, 2], [124, 133, 2], [124, 144, 2], [124, 145, 2], [124, 137, 2], [125, 134, 2], [125, 138, 2], [125, 141, 2], [125, 126, 2], [125, 127, 2], [126, 134, 2], [126, 135, 2], [126, 142, 2], [126, 145, 2], [126, 149, 2], [126, 128, 2], [127, 135, 2], [127, 136, 2], [127, 137, 2], [127, 139, 2], [127, 148, 2], [127, 149, 2], [127, 131, 2], [128, 146, 2], [128, 141, 2], [128, 140, 2], [128, 136, 2], [129, 132, 2], [129, 140, 2], [130, 136, 2], [131, 140, 2], [132, 138, 2], [133, 136, 2], [133, 139, 2], [133, 147, 2], [134, 142, 2], [134, 139, 2], [134, 143, 2], [134, 144, 2], [134, 146, 2], [135, 148, 2], [136, 149, 2], [136, 141, 2], [136, 139, 2], [136, 142, 2], [137, 139, 2], [138, 148, 2], [138, 146, 2], [140, 142, 2], [140, 149, 2], [141, 142, 2], [141, 147, 2], [142, 148, 2], [143, 148, 2], [143, 149, 2], [143, 147, 2], [145, 148, 2], [150, 151, 3], [150, 176, 3], [150, 169, 3], [150, 179, 3], [150, 173, 3], [151, 168, 3], [151, 170, 3], [151, 156, 3], [151, 174, 3], [151, 160, 3], [151, 195, 3], [151, 176, 3], [151, 179, 3], [152, 193, 3], [152, 168, 3], [152, 185, 3], [152, 171, 3], [152, 177, 3], [152, 164, 3], [152, 165, 3], [153, 166, 3], [153, 169, 3], [153, 170, 3], [153, 171, 3], [153, 172, 3], [153, 178, 3], [154, 189, 3], [154, 174, 3], [154, 175, 3], [154, 176, 3], [154, 161, 3], [154, 195, 3], [155, 182, 3], [155, 167, 3], [155, 186, 3], [155, 171, 3], [155, 157, 3], [155, 183, 3], [155, 161, 3], [156, 198, 3], [156, 193, 3], [156, 171, 3], [156, 175, 3], [156, 161, 3], [156, 164, 3], [156, 181, 3], [157, 167, 3], [157, 185, 3], [157, 170, 3], [157, 174, 3], [157, 189, 3], [157, 158, 3], [157, 159, 3], [157, 176, 3], [157, 178, 3], [157, 181, 3], [158, 184, 3], [158, 170, 3], [158, 171, 3], [158, 159, 3], [158, 192, 3], [158, 161, 3], [158, 195, 3], [158, 187, 3], [159, 166, 3], [159, 174, 3], [159, 168, 3], [159, 194, 3], [159, 192, 3], [159, 189, 3], [159, 162, 3], [159, 195, 3], [159, 180, 3], [160, 192, 3], [160, 168, 3], [160, 169, 3], [160, 186, 3], [160, 183, 3], [160, 188, 3], [160, 167, 3], [160, 176, 3], [160, 163, 3], [160, 187, 3], [161, 184, 3], [161, 192, 3], [161, 167, 3], [161, 168, 3], [161, 169, 3], [161, 171, 3], [161, 174, 3], [161, 180, 3], [162, 193, 3], [162, 164, 3], [162, 175, 3], [163, 166, 3], [163, 196, 3], [163, 180, 3], [164, 174, 3], [165, 186, 3], [165, 172, 3], [165, 174, 3], [165, 196, 3], [165, 181, 3], [166, 199, 3], [166, 191, 3], [166, 189, 3], [166, 197, 3], [167, 171, 3], [167, 179, 3], [167, 180, 3], [168, 175, 3], [168, 173, 3], [168, 179, 3], [169, 193, 3], [169, 188, 3], [169, 192, 3], [170, 174, 3], [170, 175, 3], [170, 176, 3], [170, 193, 3], [170, 179, 3], [171, 177, 3], [171, 186, 3], [171, 184, 3], [171, 195, 3], [172, 190, 3], [174, 182, 3], [174, 187, 3], [176, 190, 3], [176, 177, 3], [176, 194, 3], [176, 179, 3], [177, 188, 3], [178, 186, 3], [179, 185, 3], [179, 182, 3], [179, 196, 3], [179, 193, 3], [179, 180, 3], [180, 198, 3], [180, 195, 3], [180, 186, 3], [180, 191, 3], [180, 190, 3], [180, 197, 3], [181, 183, 3], [181, 185, 3], [181, 186, 3], [181, 190, 3], [181, 197, 3], [181, 195, 3], [181, 196, 3], [182, 198, 3], [182, 190, 3], [182, 185, 3], [182, 196, 3], [183, 189, 3], [184, 198, 3], [184, 199, 3], [184, 195, 3], [186, 197, 3], [186, 191, 3], [186, 195, 3], [187, 194, 3], [187, 197, 3], [191, 198, 3], [192, 199, 3], [192, 195, 3], [193, 199, 3], [194, 198, 3], [195, 196, 3], [195, 197, 3], [195, 198, 3], [200, 232, 4], [200, 235, 4], [200, 206, 4], [200, 224, 4], [200, 213, 4], [200, 246, 4], [201, 216, 4], [201, 224, 4], [201, 227, 4], [201, 220, 4], [201, 205, 4], [201, 222, 4], [201, 239, 4], [201, 208, 4], [201, 211, 4], [201, 247, 4], [202, 233, 4], [202, 203, 4], [202, 204, 4], [202, 237, 4], [202, 207, 4], [202, 240, 4], [202, 241, 4], [202, 245, 4], [203, 216, 4], [203, 217, 4], [203, 204, 4], [203, 237, 4], [203, 238, 4], [203, 239, 4], [203, 240, 4], [203, 220, 4], [203, 245, 4], [204, 248, 4], [204, 217, 4], [204, 236, 4], [204, 241, 4], [204, 227, 4], [204, 246, 4], [205, 219, 4], [205, 236, 4], [205, 237, 4], [205, 235, 4], [205, 217, 4], [205, 210, 4], [205, 227, 4], [205, 212, 4], [205, 213, 4], [206, 232, 4], [206, 234, 4], [206, 211, 4], [206, 236, 4], [206, 233, 4], [206, 242, 4], [206, 207, 4], [206, 244, 4], [206, 248, 4], [206, 217, 4], [206, 228, 4], [206, 230, 4], [207, 216, 4], [207, 221, 4], [207, 239, 4], [207, 243, 4], [207, 229, 4], [208, 240, 4], [208, 243, 4], [208, 238, 4], [208, 248, 4], [209, 219, 4], [209, 237, 4], [209, 240, 4], [209, 210, 4], [209, 243, 4], [209, 228, 4], [209, 229, 4], [209, 215, 4], [210, 216, 4], [210, 219, 4], [210, 223, 4], [210, 214, 4], [210, 247, 4], [211, 248, 4], [211, 218, 4], [211, 223, 4], [211, 224, 4], [212, 248, 4], [212, 218, 4], [212, 243, 4], [212, 227, 4], [212, 228, 4], [212, 214, 4], [213, 222, 4], [213, 246, 4], [213, 227, 4], [213, 229, 4], [213, 230, 4], [213, 231, 4], [214, 228, 4], [214, 237, 4], [214, 249, 4], [215, 216, 4], [215, 232, 4], [215, 217, 4], [215, 241, 4], [215, 225, 4], [215, 245, 4], [216, 236, 4], [216, 233, 4], [216, 235, 4], [216, 229, 4], [216, 246, 4], [217, 225, 4], [217, 246, 4], [218, 232, 4], [218, 233, 4], [218, 219, 4], [218, 221, 4], [218, 238, 4], [218, 224, 4], [218, 228, 4], [218, 243, 4], [218, 231, 4], [219, 227, 4], [219, 230, 4], [219, 237, 4], [220, 232, 4], [220, 248, 4], [220, 249, 4], [220, 235, 4], [220, 239, 4], [220, 246, 4], [221, 232, 4], [221, 248, 4], [221, 246, 4], [222, 236, 4], [222, 245, 4], [223, 248, 4], [223, 249, 4], [223, 236, 4], [223, 230, 4], [224, 226, 4], [225, 226, 4], [225, 236, 4], [226, 245, 4], [227, 239, 4], [227, 247, 4], [227, 229, 4], [229, 246, 4], [229, 240, 4], [229, 230, 4], [230, 237, 4], [230, 240, 4], [230, 247, 4], [230, 238, 4], [231, 232, 4], [231, 248, 4], [231, 236, 4], [231, 241, 4], [231, 242, 4], [232, 239, 4], [232, 240, 4], [232, 234, 4], [233, 249, 4], [233, 235, 4], [233, 234, 4], [234, 248, 4], [235, 241, 4], [235, 248, 4], [236, 240, 4], [236, 243, 4], [236, 237, 4], [237, 245, 4], [238, 239, 4], [239, 249, 4], [240, 245, 4], [246, 249, 4], [247, 249, 4]]
test_total_nodes = [[0, 1], [1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [6, 1], [7, 1], [8, 1], [9, 1], [10, 1], [11, 1], [12, 1], [13, 1], [14, 1], [15, 1], [16, 1], [17, 1], [18, 1], [19, 1], [20, 1], [21, 1], [22, 1], [23, 1], [24, 1], [25, 0], [26, 0], [27, 0], [28, 0], [29, 0], [30, 0], [31, 0], [32, 0], [33, 0], [34, 0], [35, 0], [36, 0], [37, 0], [38, 0], [39, 0], [40, 0], [41, 0], [42, 0], [43, 0], [44, 0], [45, 0], [46, 0], [47, 0], [48, 0], [49, 0], [50, 1], [51, 1], [52, 1], [53, 1], [54, 1], [55, 1], [56, 1], [57, 1], [58, 1], [59, 1], [60, 1], [61, 1], [62, 1], [63, 1], [64, 1], [65, 1], [66, 1], [67, 1], [68, 1], [69, 1], [70, 1], [71, 1], [72, 1], [73, 1], [74, 1], [75, 0], [76, 0], [77, 0], [78, 0], [79, 0], [80, 0], [81, 0], [82, 0], [83, 0], [84, 0], [85, 0], [86, 0], [87, 0], [88, 0], [89, 0], [90, 0], [91, 0], [92, 0], [93, 0], [94, 0], [95, 0], [96, 0], [97, 0], [98, 0], [99, 0], [100, 1], [101, 1], [102, 1], [103, 1], [104, 1], [105, 1], [106, 1], [107, 1], [108, 1], [109, 1], [110, 1], [111, 1], [112, 1], [113, 1], [114, 1], [115, 1], [116, 1], [117, 1], [118, 1], [119, 1], [120, 1], [121, 1], [122, 1], [123, 1], [124, 1], [125, 0], [126, 0], [127, 0], [128, 0], [129, 0], [130, 0], [131, 0], [132, 0], [133, 0], [134, 0], [135, 0], [136, 0], [137, 0], [138, 0], [139, 0], [140, 0], [141, 0], [142, 0], [143, 0], [144, 0], [145, 0], [146, 0], [147, 0], [148, 0], [149, 0], [150, 1], [151, 1], [152, 1], [153, 1], [154, 1], [155, 1], [156, 1], [157, 1], [158, 1], [159, 1], [160, 1], [161, 1], [162, 1], [163, 1], [164, 1], [165, 1], [166, 1], [167, 1], [168, 1], [169, 1], [170, 1], [171, 1], [172, 1], [173, 1], [174, 1], [175, 0], [176, 0], [177, 0], [178, 0], [179, 0], [180, 0], [181, 0], [182, 0], [183, 0], [184, 0], [185, 0], [186, 0], [187, 0], [188, 0], [189, 0], [190, 0], [191, 0], [192, 0], [193, 0], [194, 0], [195, 0], [196, 0], [197, 0], [198, 0], [199, 0], [200, 1], [201, 1], [202, 1], [203, 1], [204, 1], [205, 1], [206, 1], [207, 1], [208, 1], [209, 1], [210, 1], [211, 1], [212, 1], [213, 1], [214, 1], [215, 1], [216, 1], [217, 1], [218, 1], [219, 1], [220, 1], [221, 1], [222, 1], [223, 1], [224, 1], [225, 0], [226, 0], [227, 0], [228, 0], [229, 0], [230, 0], [231, 0], [232, 0], [233, 0], [234, 0], [235, 0], [236, 0], [237, 0], [238, 0], [239, 0], [240, 0], [241, 0], [242, 0], [243, 0], [244, 0], [245, 0], [246, 0], [247, 0], [248, 0], [249, 0]]
test_total_labels = [[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]


In [30]:
E_tot_test = np.asarray(test_total_edges)
N_tot_test = np.asarray(test_total_nodes)
labels_test = np.asarray(test_total_labels)

In [31]:
inp_test, arcnode_test, graphnode_test = gnn_utils.from_EN_to_GNN(E_tot_test, N_tot_test)

In [32]:
met = g.Evaluate(inp_test, arcnode_test, labels_test)
met

[1.0]

In [41]:
pr = g.Predict(inp, arcnode, labels)
print(type(pr))
np.savetxt("test_predictions.txt", pr, fmt='%s')

<class 'tuple'>


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
